In [23]:
pip install --upgrade snowflake-connector-python

Note: you may need to restart the kernel to use updated packages.


In [24]:
import snowflake.connector

In [25]:
conn = snowflake.connector.connect(
    user= 'mnonog',
    password = 'KayaNatinToL0RD!',
    account='cejmwpt-djb91267'
    )

#https://cejmwpt-djb91267.snowflakecomputing.com

In [26]:
with conn.cursor() as cs:
    # warehouse + db/schema setup
    cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")
    cs.execute("CREATE DATABASE IF NOT EXISTS testdb")
    cs.execute("USE DATABASE testdb")          # 👈 set current DB
    cs.execute("CREATE SCHEMA IF NOT EXISTS testschema")
    cs.execute( "CREATE OR REPLACE TABLE "    
             "test_table(col1 integer, col2 string)")
    cs.execute("INSERT INTO test_table(col1, col2) "
    		"VALUES (123, 'test string1'), (456, 'test string2')")
    
    #  stages 
    cs.execute("CREATE STAGE IF NOT EXISTS purchases_stage")
    cs.execute("CREATE STAGE IF NOT EXISTS invoices_stage")
    cs.execute("CREATE STAGE IF NOT EXISTS supplier_stage")
    
    # File formats
    cs.execute("""
        CREATE OR REPLACE FILE FORMAT csv_ff 
        TYPE = CSV 
        FIELD_OPTIONALLY_ENCLOSED_BY = '"'
        SKIP_HEADER = 1 
        NULL_IF = ('', 'NULL')
    """)
    
    cs.execute("""
        CREATE OR REPLACE FILE FORMAT xml_ff 
        TYPE = XML 
        STRIP_OUTER_ELEMENT = TRUE
    """)
    # cs.execute('SELECT * FROM test_table')
    # print(cs.fetchmany(2))
conn.close()



In [27]:
%pip install psycopg2-binary pandas lxml python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import snowflake.connector as sf


WH, DB, SC = "ETL_WH", "ETL_DB", "ETL_SCHEMA"
BASE_DIR = Path(r"/home/jovyan/Desktop/SQL/SQL_FINAL_PROJECT/Data-5/Monthly PO Data").resolve()
OUT_CSV  = BASE_DIR / "combined_purchases.csv"   # 统一输出文件

# 2) 需要保留并标准化到 Snowflake 的列（目标列名）
TARGET_COLS = [
    "PurchaseOrderID",
    "PurchaseOrderLineID",
    "ReceivedOuters",
    "ExpectedUnitPricePerOuter",
    "OrderDate",
    "SupplierID",
]

# 3) 列名别名表（尽量覆盖你文件里的各种写法；不够的话再加几种）
ALIASES = {
    "PurchaseOrderID":           ["purchaseorderid", "purchase_order_id", "poid", "orderid"],
    "PurchaseOrderLineID":       ["purchaseorderlineid", "purchase_order_line_id", "polineid", "orderlineid"],
    "ReceivedOuters":            ["receivedouters", "received_outers", "receivedoutersqty", "receivedoutersquantity", "received_qty"],
    "ExpectedUnitPricePerOuter": ["expectedunitpriceperouter", "expected_unit_price_per_outer", "unitpriceperouter", "expectedprice", "unitprice"],
    "OrderDate":                 ["orderdate", "order_date", "date", "order_dt"],
    "SupplierID":                ["supplierid", "supplier_id", "vendorid", "vendor_id"],
}

def norm(name: str) -> str:
    """规范化列名：只保留字母数字并小写，便于匹配"""
    return "".join(ch.lower() for ch in name if ch.isalnum())

def pick_and_rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    """从原始 df 里按别名表挑出需要的列，并重命名为标准列名"""
    orig_to_norm = {c: norm(c) for c in df.columns}
    norm_to_orig = {v: k for k, v in orig_to_norm.items()}  # 取第一个出现的映射即可

    selected = {}
    missing  = []
    for tgt, alias_list in ALIASES.items():
        found = None
        for alias in alias_list:
            if alias in norm_to_orig:
                found = norm_to_orig[alias]
                break
        if found is None:
            missing.append(tgt)
        else:
            selected[tgt] = found

    if missing:
        raise ValueError(f"缺少必须列：{missing}; 文件列={list(df.columns)}")


    out = df[[selected[c] for c in TARGET_COLS]].copy()
    out.columns = TARGET_COLS
    return out

frames = []
csv_files = sorted(BASE_DIR.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"在 {BASE_DIR} 下没有找到 .csv 文件")

for p in csv_files:
    df = pd.read_csv(p, dtype=str, keep_default_na=False, na_values=["", "NULL"])
    df_std = pick_and_rename_columns(df)


    for c in ["ReceivedOuters", "ExpectedUnitPricePerOuter"]:
        df_std[c] = pd.to_numeric(df_std[c], errors="coerce")

    df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)


    df_std = df_std.dropna(subset=["PurchaseOrderID", "PurchaseOrderLineID", "OrderDate"])

    df_std["ReceivedOuters"] = df_std["ReceivedOuters"].fillna(0)
    df_std["ExpectedUnitPricePerOuter"] = df_std["ExpectedUnitPricePerOuter"].fillna(0)

    frames.append(df_std)

combined = pd.concat(frames, ignore_index=True)

combined.to_csv(OUT_CSV, index=False)
print(f"combine to 1：{OUT_CSV} line={len(combined):,}")

conn = sf.connect(
    user=os.getenv("SNOW_USER", "ETHANAN2000"),
    password=os.getenv("SNOW_PASSWORD", "An67087833@123"),
    account=os.getenv("SNOW_ACCOUNT", "svogymj-bxb71103") 
)


/tmp/ipykernel_93494/868590347.py:75: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_93494/868590347.py:75: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_93494/868590347.py:75: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, se

combine to 1：/home/jovyan/Desktop/SQL/SQL_FINAL_PROJECT/Data-5/Monthly PO Data/combined_purchases.csv line=229,124


In [29]:
cs = conn.cursor()

WH, DB, SC = "ETL_WH", "ETL_DB", "ETL_SCHEMA"
cs.execute(f"CREATE WAREHOUSE IF NOT EXISTS {WH} WAREHOUSE_SIZE=SMALL AUTO_SUSPEND=60 AUTO_RESUME=TRUE")
cs.execute(f"CREATE DATABASE  IF NOT EXISTS {DB}")
cs.execute(f"CREATE SCHEMA    IF NOT EXISTS {DB}.{SC}")
cs.execute(f"USE WAREHOUSE {WH}")
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SC}")


In [30]:

cs.execute("CREATE STAGE IF NOT EXISTS purchases_stage")
cs.execute("""
CREATE OR REPLACE FILE FORMAT csv_ff
  TYPE = CSV
  FIELD_OPTIONALLY_ENCLOSED_BY = '\"'
  PARSE_HEADER = TRUE
  NULL_IF = ('','NULL')
  TRIM_SPACE = TRUE
""")


In [31]:

cs.execute("""
CREATE OR REPLACE TABLE purchases_detail (
  PurchaseOrderID           STRING,
  PurchaseOrderLineID       STRING,
  ReceivedOuters            NUMBER(18,4),
  ExpectedUnitPricePerOuter NUMBER(18,4),
  OrderDate                 DATE,
  SupplierID                STRING
)
""")


In [32]:

cs.execute(f"PUT 'file:///{OUT_CSV.as_posix()}' @purchases_stage AUTO_COMPRESS=TRUE OVERWRITE=TRUE")

cs.execute("ALTER SESSION SET DATE_INPUT_FORMAT='AUTO'")
cs.execute("""
COPY INTO purchases_detail
FROM @purchases_stage
FILE_FORMAT=(FORMAT_NAME='csv_ff')
MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
ON_ERROR='ABORT_STATEMENT'
""")


# 10) 校验
cs.execute("SELECT COUNT(*) FROM purchases_detail")
print("Snowflake 行数:", cs.fetchone()[0])
cs.execute("SELECT * FROM purchases_detail ORDER BY OrderDate, PurchaseOrderID LIMIT 5")
for row in cs.fetchall():
    print(row)

cs.close(); conn.close()


Snowflake 行数: 229124
('1', '1', Decimal('18.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '2', Decimal('21.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '3', Decimal('18.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '2', Decimal('21.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '1', Decimal('18.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')


# PART 2

## Setting Up the Warehouse and Database

In [34]:
import os
import snowflake.connector as sf



# === 建议用环境变量/ .env ===
SNOW_USER=os.getenv("SNOW_USER","<mnonog>")
SNOW_PASSWORD=os.getenv("SNOW_PASSWORD","<KayaNatinToL0RD!>")
SNOW_ACCOUNT=os.getenv("SNOW_ACCOUNT","cejmwpt-djb91267")   # 只写标识，不要写 https://...
SNOW_ROLE=os.getenv("SNOW_ROLE","ACCOUNTADMIN")
WH_NAME=os.getenv("SNOW_WH","ETL_WH")
DB_NAME=os.getenv("SNOW_DB","ETL_DB")
SCHEMA_NAME=os.getenv("SNOW_SCHEMA","ETL_SCHEMA")

conn = sf.connect(user='mnonog', password='KayaNatinToL0RD!', account='cejmwpt-djb91267')
cs = conn.cursor()

# 基础对象
cs.execute(f"CREATE WAREHOUSE IF NOT EXISTS {WH_NAME} WAREHOUSE_SIZE=SMALL AUTO_SUSPEND=60 AUTO_RESUME=TRUE")
cs.execute(f"CREATE DATABASE  IF NOT EXISTS {DB_NAME}")
cs.execute(f"CREATE SCHEMA    IF NOT EXISTS {DB_NAME}.{SCHEMA_NAME}")
cs.execute(f"USE ROLE {SNOW_ROLE}")
cs.execute(f"USE WAREHOUSE {WH_NAME}")
cs.execute(f"USE DATABASE {DB_NAME}")
cs.execute(f"USE SCHEMA {SCHEMA_NAME}")

# 统一 stage / 文件格式
cs.execute("CREATE STAGE IF NOT EXISTS purchases_stage")
cs.execute("CREATE STAGE IF NOT EXISTS invoices_stage")
cs.execute("CREATE STAGE IF NOT EXISTS supplier_stage")

cs.execute("""CREATE OR REPLACE FILE FORMAT csv_ff 
  TYPE=CSV FIELD_OPTIONALLY_ENCLOSED_BY='\"' SKIP_HEADER=1 NULL_IF=('','NULL')""")
cs.execute("""CREATE OR REPLACE FILE FORMAT xml_ff TYPE=XML STRIP_OUTER_ELEMENT=TRUE""")


## Connecting to Snowflakes

In [35]:
conn = sf.connect(
    user=os.getenv("SNOW_USER", "mnonog"),
    password=os.getenv("SNOW_PASSWORD", "KayaNatinToL0RD!"),
    account=os.getenv("SNOW_ACCOUNT", "cejmwpt-djb91267")  # 只写标识，不要 https:// 前缀
)
cs = conn.cursor()

## Part 2: Task 1 and 2: Creating Single Table of Purchases Data and Adding the POAmount Column

In [36]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import snowflake.connector as sf


WH, DB, SC = "ETL_WH", "ETL_DB", "ETL_SCHEMA"
BASE_DIR = Path(r"/home/jovyan/Desktop/SQL/SQL_FINAL_PROJECT/Data-5/Monthly PO Data").resolve()
OUT_CSV  = BASE_DIR / "combined_purchases.csv"   # 统一输出文件

# 2) 需要保留并标准化到 Snowflake 的列（目标列名）
TARGET_COLS = [
    "PurchaseOrderID",
    "PurchaseOrderLineID",
    "ReceivedOuters",
    "ExpectedUnitPricePerOuter",
    "OrderDate",
    "SupplierID",
]

# 3) 列名别名表（尽量覆盖你文件里的各种写法；不够的话再加几种）
ALIASES = {
    "PurchaseOrderID":           ["purchaseorderid", "purchase_order_id", "poid", "orderid"],
    "PurchaseOrderLineID":       ["purchaseorderlineid", "purchase_order_line_id", "polineid", "orderlineid"],
    "ReceivedOuters":            ["receivedouters", "received_outers", "receivedoutersqty", "receivedoutersquantity", "received_qty"],
    "ExpectedUnitPricePerOuter": ["expectedunitpriceperouter", "expected_unit_price_per_outer", "unitpriceperouter", "expectedprice", "unitprice"],
    "OrderDate":                 ["orderdate", "order_date", "date", "order_dt"],
    "SupplierID":                ["supplierid", "supplier_id", "vendorid", "vendor_id"],
}

def norm(name: str) -> str:
    return "".join(ch.lower() for ch in name if ch.isalnum())

def pick_and_rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    orig_to_norm = {c: norm(c) for c in df.columns}
    norm_to_orig = {v: k for k, v in orig_to_norm.items()}  # 取第一个出现的映射即可

    selected = {}
    missing  = []
    for tgt, alias_list in ALIASES.items():
        found = None
        for alias in alias_list:
            if alias in norm_to_orig:
                found = norm_to_orig[alias]
                break
        if found is None:
            missing.append(tgt)
        else:
            selected[tgt] = found

    if missing:
        raise ValueError(f"缺少必须列：{missing}; 文件列={list(df.columns)}")


    out = df[[selected[c] for c in TARGET_COLS]].copy()
    out.columns = TARGET_COLS
    return out

frames = []
csv_files = sorted(BASE_DIR.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"在 {BASE_DIR} 下没有找到 .csv 文件")

for p in csv_files:
    df = pd.read_csv(p, dtype=str, keep_default_na=False, na_values=["", "NULL"])
    df_std = pick_and_rename_columns(df)


    for c in ["ReceivedOuters", "ExpectedUnitPricePerOuter"]:
        df_std[c] = pd.to_numeric(df_std[c], errors="coerce")

    df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)


    df_std = df_std.dropna(subset=["PurchaseOrderID", "PurchaseOrderLineID", "OrderDate"])

    df_std["ReceivedOuters"] = df_std["ReceivedOuters"].fillna(0)
    df_std["ExpectedUnitPricePerOuter"] = df_std["ExpectedUnitPricePerOuter"].fillna(0)

    frames.append(df_std)

combined = pd.concat(frames, ignore_index=True)

combined.to_csv(OUT_CSV, index=False)
print(f"combine to 1：{OUT_CSV} line={len(combined):,}")

conn = sf.connect(
    user=os.getenv("SNOW_USER", "mnonog"),
    password=os.getenv("SNOW_PASSWORD", "KayaNatinToL0RD!"),
    account=os.getenv("SNOW_ACCOUNT", "cejmwpt-djb91267") 
)

cs = conn.cursor()

WH, DB, SC = "ETL_WH", "ETL_DB", "ETL_SCHEMA"
cs.execute(f"CREATE WAREHOUSE IF NOT EXISTS {WH} WAREHOUSE_SIZE=SMALL AUTO_SUSPEND=60 AUTO_RESUME=TRUE")
cs.execute(f"CREATE DATABASE  IF NOT EXISTS {DB}")
cs.execute(f"CREATE SCHEMA    IF NOT EXISTS {DB}.{SC}")
cs.execute(f"USE WAREHOUSE {WH}")
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SC}")

cs.execute("CREATE STAGE IF NOT EXISTS purchases_stage")
cs.execute("""
CREATE OR REPLACE FILE FORMAT csv_ff
  TYPE = CSV
  FIELD_OPTIONALLY_ENCLOSED_BY = '\"'
  PARSE_HEADER = TRUE
  NULL_IF = ('','NULL')
  TRIM_SPACE = TRUE
""")

cs.execute("""
CREATE OR REPLACE TABLE purchases_detail (
  PurchaseOrderID           STRING,
  PurchaseOrderLineID       STRING,
  ReceivedOuters            NUMBER(18,4),
  ExpectedUnitPricePerOuter NUMBER(18,4),
  OrderDate                 DATE,
  SupplierID                STRING
)
""")



cs.execute(f"PUT 'file:///{OUT_CSV.as_posix()}' @purchases_stage AUTO_COMPRESS=TRUE OVERWRITE=TRUE")

cs.execute("ALTER SESSION SET DATE_INPUT_FORMAT='AUTO'")
cs.execute("""
COPY INTO purchases_detail
FROM @purchases_stage
FILE_FORMAT=(FORMAT_NAME='csv_ff')
MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
ON_ERROR='ABORT_STATEMENT'
""")

# 10) 校验
cs.execute("SELECT COUNT(*) FROM purchases_detail")
print("Snowflake 行数:", cs.fetchone()[0])
cs.execute("SELECT * FROM purchases_detail ORDER BY OrderDate, PurchaseOrderID LIMIT 5")
for row in cs.fetchall():
    print(row)
  
    
# line-level view
cs.execute("""
CREATE OR REPLACE VIEW PurchaseOrderTotals AS
SELECT
  PurchaseOrderID,
  PurchaseOrderLineID,
  ReceivedOuters,
  ExpectedUnitPricePerOuter,
  OrderDate,
  SupplierID,
  SUM(ReceivedOuters * ExpectedUnitPricePerOuter)
    OVER (PARTITION BY PurchaseOrderID) AS POAmount
FROM purchases_detail
ORDER BY PurchaseOrderID, PurchaseOrderLineID
""")

# add column (no-op if it already exists; remove OR REPLACE since Snowflake doesn't support it on columns)
cs.execute("ALTER TABLE purchases_detail ADD COLUMN POAmount NUMBER(18,2)")

# backfill totals (same POAmount for all lines of the same order)
cs.execute("""
UPDATE purchases_detail AS pd
SET POAmount = t.POAmount
FROM (
  SELECT
    PurchaseOrderID,
    ROUND(SUM(ReceivedOuters * ExpectedUnitPricePerOuter), 2) AS POAmount
  FROM purchases_detail
  GROUP BY PurchaseOrderID
) AS t
WHERE pd.PurchaseOrderID = t.PurchaseOrderID
""")



/tmp/ipykernel_93494/4052790598.py:73: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_93494/4052790598.py:73: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_93494/4052790598.py:73: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default,

combine to 1：/home/jovyan/Desktop/SQL/SQL_FINAL_PROJECT/Data-5/Monthly PO Data/combined_purchases.csv line=237,307
Snowflake 行数: 237307
('1', '1', Decimal('18.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '2', Decimal('21.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '3', Decimal('18.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '2', Decimal('21.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')
('1', '1', Decimal('18.0000'), Decimal('5.5000'), datetime.date(2019, 1, 1), '2')


## Part 2 Task 3

In [37]:
# === 3) Extract & load supplier invoice XML (one row per invoice) ===

# File format + stage (idempotent)
cs.execute("CREATE STAGE IF NOT EXISTS invoices_stage")
cs.execute("""
CREATE OR REPLACE FILE FORMAT xml_ff
  TYPE = XML
  STRIP_OUTER_ELEMENT = TRUE
""")

# Your XML file path
XML_FILE = Path("/home/jovyan/Desktop/SQL/SQL_FINAL_PROJECT/Data-5/Supplier Transactions XML.xml").resolve()

# Keep XML isolated in a subfolder
cs.execute(f"PUT 'file:///{XML_FILE.as_posix()}' @invoices_stage/incoming AUTO_COMPRESS=TRUE OVERWRITE=TRUE")

# (optional) sanity check
cs.execute("LIST @invoices_stage/incoming"); print(cs.fetchall())

# 1) Land raw XML into a VARIANT table (simple COPY is required)
cs.execute("""
CREATE OR REPLACE TABLE supplier_invoices_raw (
  doc VARIANT,
  sourcefile STRING
)
""")

cs.execute("""
COPY INTO supplier_invoices_raw (doc, sourcefile)
FROM (
  SELECT $1, METADATA$FILENAME
  FROM @invoices_stage/incoming (FILE_FORMAT => 'xml_ff')
)
ON_ERROR='ABORT_STATEMENT'
""")

# 2) Final relational table (one row = one invoice)
cs.execute("""
CREATE OR REPLACE TABLE supplier_invoices (
  SupplierTransactionID NUMBER,
  SupplierID            NUMBER,
  TransactionTypeID     NUMBER,
  PurchaseOrderID       NUMBER,
  PaymentMethodID       NUMBER,
  SupplierInvoiceNumber STRING,
  TransactionDate       DATE,
  AmountExcludingTax    NUMBER(18,2),
  TaxAmount             NUMBER(18,2),
  TransactionAmount     NUMBER(18,2),
  OutstandingBalance    NUMBER(18,2),
  FinalizationDate      DATE,
  IsFinalized           BOOLEAN,
  LastEditedBy          NUMBER,
  SourceFile            STRING
)
""")

cs.execute("ALTER SESSION SET DATE_INPUT_FORMAT='AUTO', TIMESTAMP_INPUT_FORMAT='AUTO'")

# 3) Shred XML from RAW -> FINAL using the hinted functions
#    (handles either <root><row>...</row></root> OR already-row-level documents)
cs.execute("""
INSERT INTO supplier_invoices
SELECT
  TRY_TO_NUMBER(GET(XMLGET(f.value,'SupplierTransactionID'), '$')::string)              AS SupplierTransactionID,
  TRY_TO_NUMBER(GET(XMLGET(f.value,'SupplierID'), '$')::string)                         AS SupplierID,
  TRY_TO_NUMBER(GET(XMLGET(f.value,'TransactionTypeID'), '$')::string)                  AS TransactionTypeID,
  TRY_TO_NUMBER(GET(XMLGET(f.value,'PurchaseOrderID'), '$')::string)                    AS PurchaseOrderID,
  TRY_TO_NUMBER(GET(XMLGET(f.value,'PaymentMethodID'), '$')::string)                    AS PaymentMethodID,
  XMLGET(f.value,'SupplierInvoiceNumber'):"$"::string                                    AS SupplierInvoiceNumber,
  TRY_TO_DATE(XMLGET(f.value,'TransactionDate'):"$"::string)                            AS TransactionDate,
  TRY_TO_NUMBER(REPLACE(XMLGET(f.value,'AmountExcludingTax'):"$"::string, ',', ''),38,2) AS AmountExcludingTax,
  TRY_TO_NUMBER(REPLACE(XMLGET(f.value,'TaxAmount'):"$"::string, ',', ''),38,2)          AS TaxAmount,
  TRY_TO_NUMBER(REPLACE(XMLGET(f.value,'TransactionAmount'):"$"::string, ',', ''),38,2)  AS TransactionAmount,
  TRY_TO_NUMBER(REPLACE(XMLGET(f.value,'OutstandingBalance'):"$"::string, ',', ''),38,2) AS OutstandingBalance,
  TRY_TO_DATE(XMLGET(f.value,'FinalizationDate'):"$"::string)                           AS FinalizationDate,
  IFF(LOWER(NULLIF(XMLGET(f.value,'IsFinalized'):"$"::string,'')) IN ('1','true','yes'),
      TRUE, FALSE)                                                                       AS IsFinalized,
  TRY_TO_NUMBER(XMLGET(f.value,'LastEditedBy'):"$"::string)                             AS LastEditedBy,
  r.sourcefile                                                                           AS SourceFile
FROM supplier_invoices_raw r,
     LATERAL FLATTEN(
       input => IFF(IS_ARRAY(r.doc:root.row), r.doc:root.row, ARRAY_CONSTRUCT(r.doc))
     ) f
""")

# verify
cs.execute("SELECT COUNT(*) FROM supplier_invoices")
print("supplier_invoices rows:", cs.fetchone()[0])
cs.execute("SELECT * FROM supplier_invoices ORDER BY TransactionDate, SupplierTransactionID LIMIT 5")
for r in cs.fetchall():
    print(r)


[('invoices_stage/incoming/Supplier Transactions XML.xml.gz', 72528, 'c1f7dea87b23878052a0201ed1c68223', 'Thu, 11 Sep 2025 05:09:34 GMT')]
supplier_invoices rows: 2438
(134, 2, 5, 1, 4, '7290', datetime.date(2019, 1, 2), Decimal('313.50'), Decimal('47.03'), Decimal('360.53'), Decimal('0.00'), datetime.date(2019, 1, 7), True, 4, 'incoming/Supplier Transactions XML.xml.gz')
(169, 4, 5, 2, 4, '3898', datetime.date(2019, 1, 2), Decimal('21732.00'), Decimal('3259.80'), Decimal('24991.80'), Decimal('0.00'), datetime.date(2019, 1, 7), True, 4, 'incoming/Supplier Transactions XML.xml.gz')
(186, 5, 5, 3, 4, '616', datetime.date(2019, 1, 2), Decimal('2740.50'), Decimal('411.11'), Decimal('3151.61'), Decimal('0.00'), datetime.date(2019, 1, 7), True, 4, 'incoming/Supplier Transactions XML.xml.gz')
(215, 7, 5, 4, 4, '3869', datetime.date(2019, 1, 2), Decimal('42481.20'), Decimal('6372.19'), Decimal('48853.39'), Decimal('0.00'), datetime.date(2019, 1, 7), True, 4, 'incoming/Supplier Transactions XML

## Part 2 Task 4

In [38]:
cs.execute(f"""
CREATE OR REPLACE VIEW {DB}.{SC}.po_invoice_join_vw AS
SELECT
  d.PurchaseOrderID,
  d.PurchaseOrderLineID,
  d.SupplierID,
  d.OrderDate,
  d.ReceivedOuters,
  d.ExpectedUnitPricePerOuter,
  i.SupplierTransactionID,
  i.SupplierInvoiceNumber,
  i.TransactionDate   AS InvoiceDate,
  i.TransactionAmount AS InvoiceAmount
FROM {DB}.{SC}.purchases_detail d
JOIN {DB}.{SC}.supplier_invoices i
  ON TRY_TO_NUMBER(d.PurchaseOrderID) = i.PurchaseOrderID
""")


## Part 2 Task 5

In [39]:
# === 5) purchase_orders_and_invoices (MV if possible, else TABLE) ===

# 5.1 Build a PO header total at the order level (typed as NUMBER to match invoices)
cs.execute(f"""
CREATE OR REPLACE TABLE {DB}.{SC}.po_header_totals_tmp AS
SELECT
  TRY_TO_NUMBER(PurchaseOrderID) AS PurchaseOrderID,
  TRY_TO_NUMBER(SupplierID)      AS SupplierID,
  MIN(OrderDate)                 AS OrderDate,
  ROUND(SUM(ReceivedOuters * ExpectedUnitPricePerOuter), 2) AS POAmount
FROM {DB}.{SC}.purchases_detail
GROUP BY 1,2
""")

# 5.2 Attempt Materialized View (fallback to TABLE if MV not supported for this join)
create_mv_sql = f"""
CREATE OR REPLACE MATERIALIZED VIEW {DB}.{SC}.purchase_orders_and_invoices AS
SELECT
  p.PurchaseOrderID,
  p.SupplierID,
  p.OrderDate,
  p.POAmount,
  i.SupplierTransactionID,
  i.SupplierInvoiceNumber,
  i.TransactionDate      AS InvoiceDate,
  i.AmountExcludingTax,
  i.TransactionAmount,
  ROUND(i.AmountExcludingTax - p.POAmount, 2) AS invoiced_vs_quoted
FROM {DB}.{SC}.po_header_totals_tmp p
JOIN {DB}.{SC}.supplier_invoices i
  ON p.PurchaseOrderID = i.PurchaseOrderID
 AND p.SupplierID      = i.SupplierID
"""

create_tbl_sql = f"""
CREATE OR REPLACE TABLE {DB}.{SC}.purchase_orders_and_invoices AS
SELECT
  p.PurchaseOrderID,
  p.SupplierID,
  p.OrderDate,
  p.POAmount,
  i.SupplierTransactionID,
  i.SupplierInvoiceNumber,
  i.TransactionDate      AS InvoiceDate,
  i.AmountExcludingTax,
  i.TransactionAmount,
  ROUND(i.AmountExcludingTax - p.POAmount, 2) AS invoiced_vs_quoted
FROM {DB}.{SC}.po_header_totals_tmp p
JOIN {DB}.{SC}.supplier_invoices i
  ON p.PurchaseOrderID = i.PurchaseOrderID
 AND p.SupplierID      = i.SupplierID
"""

try:
    cs.execute(create_mv_sql)
    print("Created MATERIALIZED VIEW:", f"{DB}.{SC}.purchase_orders_and_invoices")
except Exception as e:
    print("Materialized view not supported for this definition; creating TABLE instead. Reason:", str(e)[:140])
    cs.execute(create_tbl_sql)
    print("Created TABLE:", f"{DB}.{SC}.purchase_orders_and_invoices")

# 5.3 Quick sanity checks
cs.execute(f"SELECT COUNT(*) FROM {DB}.{SC}.purchase_orders_and_invoices")
print("Rows in purchase_orders_and_invoices:", cs.fetchone()[0])

cs.execute(f"""
SELECT PurchaseOrderID, SupplierID, POAmount, AmountExcludingTax, invoiced_vs_quoted
FROM {DB}.{SC}.purchase_orders_and_invoices
ORDER BY PurchaseOrderID
LIMIT 10
""")
for r in cs.fetchall():
    print(r)


Materialized view not supported for this definition; creating TABLE instead. Reason: 001998 (42710): SQL compilation error:
Object 'PURCHASE_ORDERS_AND_INVOICES' already exists as TABLE
Created TABLE: ETL_DB.ETL_SCHEMA.purchase_orders_and_invoices
Rows in purchase_orders_and_invoices: 2023
(1, 2, Decimal('9091.50'), Decimal('313.50'), Decimal('-8778.00'))
(2, 4, Decimal('630228.00'), Decimal('21732.00'), Decimal('-608496.00'))
(3, 5, Decimal('79474.50'), Decimal('2740.50'), Decimal('-76734.00'))
(4, 7, Decimal('1231954.80'), Decimal('42481.20'), Decimal('-1189473.60'))
(5, 10, Decimal('1016957.50'), Decimal('35067.50'), Decimal('-981890.00'))
(6, 12, Decimal('160326.50'), Decimal('5528.50'), Decimal('-154798.00'))
(7, 4, Decimal('290014.50'), Decimal('10000.50'), Decimal('-280014.00'))
(8, 5, Decimal('19053.00'), Decimal('657.00'), Decimal('-18396.00'))
(9, 7, Decimal('269163.50'), Decimal('9281.50'), Decimal('-259882.00'))
(10, 10, Decimal('30087.50'), Decimal('1037.50'), Decimal('-29

## Part 2 Task 6

In [ ]:
# === 6) Extract supplier_case from Postgres → local CSV → Snowflake stage ===
# Requires: pip install psycopg2-binary

import psycopg2, re
from pathlib import Path

# ---- Postgres connection (env or defaults) ----
PGHOST = os.getenv("PGHOST", "localhost")
PGPORT = int(os.getenv("PGPORT", "5432"))
PGUSER = os.getenv("PGUSER", "postgres")
PGPASSWORD = os.getenv("PGPASSWORD", "postgres")
PGDATABASE = os.getenv("PGDATABASE", "postgres")

EXPORT_DIR = Path("/home/jovyan/Desktop/SQL/SQL_FINAL_PROJECT/Data-5").resolve()
EXPORT_DIR.mkdir(parents=True, exist_ok=True)
PG_CSV = EXPORT_DIR / "supplier_case.csv"

# 6a) Postgres → CSV (server-side COPY to client, no pandas DataFrame)
with psycopg2.connect(
    host=PGHOST, port=PGPORT, user=PGUSER, password=PGPASSWORD, dbname=PGDATABASE
) as pg_conn, pg_conn.cursor() as cur, open(PG_CSV, "w", newline="", encoding="utf-8") as f:
    # If your table is in a schema, qualify it: schema_name.supplier_case
    cur.copy_expert("COPY supplier_case TO STDOUT WITH CSV HEADER", f)

print(f"Exported Postgres supplier_case -> {PG_CSV}")

# 6b) Push CSV into Snowflake stage (keep it tidy in a subfolder)
cs.execute(f"CREATE STAGE IF NOT EXISTS {DB}.{SC}.supplier_stage")
cs.execute(f"PUT 'file:///{PG_CSV.as_posix()}' @{DB}.{SC}.supplier_stage/pg_export AUTO_COMPRESS=TRUE OVERWRITE=TRUE")
cs.execute(f"LIST @{DB}.{SC}.supplier_stage/pg_export")
print(cs.fetchall())

# ---- Helper: infer Snowflake column types from CSV sample ----
import csv
from datetime import datetime

_int_re   = re.compile(r"^[+-]?\d+$")
_float_re = re.compile(r"^[+-]?\d*\.\d+$")
_bool_set = {"true","false","t","f","1","0","yes","no"}

def _looks_bool(v:str)->bool:
    return v.lower() in _bool_set

def _looks_int(v:str)->bool:
    return _int_re.match(v) is not None

def _looks_float(v:str)->bool:
    return _float_re.match(v) is not None

def _looks_date(v:str)->bool:
    # let Snowflake parse flexibly; we only do a quick sniff
    for fmt in ("%Y-%m-%d","%m/%d/%Y","%Y/%m/%d"):
        try:
            datetime.strptime(v, fmt); return True
        except: pass
    return False

def _looks_ts(v:str)->bool:
    for fmt in ("%Y-%m-%d %H:%M:%S","%Y-%m-%dT%H:%M:%S","%m/%d/%Y %H:%M:%S"):
        try:
            datetime.strptime(v, fmt); return True
        except: pass
    return False

def generate_snowflake_ddl(csv_path: Path, fq_table: str, sample_rows:int=2000)->str:
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        cols = reader.fieldnames
        # flags per column
        flags = {c: {"bool":True,"int":True,"float":True,"date":True,"ts":True,"scale":0} for c in cols}
        n=0
        for row in reader:
            n += 1
            for c in cols:
                v = (row[c] or "").strip()
                if v=="":
                    continue
                if not _looks_bool(v):  flags[c]["bool"]=False
                if _looks_int(v):
                    pass
                else:
                    flags[c]["int"]=False
                if _looks_float(v):
                    # track decimal scale
                    try:
                        sc = len(v.split(".")[1])
                        flags[c]["scale"] = max(flags[c]["scale"], sc)
                    except: pass
                else:
                    flags[c]["float"]=False
                if not _looks_date(v): flags[c]["date"]=False
                if not _looks_ts(v):   flags[c]["ts"]=False
            if n>=sample_rows: break

    def pick_type(fl):
        if fl["ts"]:   return "TIMESTAMP_NTZ"
        if fl["date"]: return "DATE"
        if fl["bool"]: return "BOOLEAN"
        if fl["int"]:  return "NUMBER(38,0)"
        if fl["float"]:
            sc = min(max(fl["scale"], 2), 9)
            return f"NUMBER(38,{sc})"
        return "STRING"

    # quote identifiers safely (handle spaces/mixed case)
    def qident(name:str)->str:
        return '"' + name.replace('"','""') + '"'

    cols_sql = ",\n  ".join(f"{qident(c)} {pick_type(flags[c])}" for c in cols)
    return f"CREATE OR REPLACE TABLE {fq_table} (\n  {cols_sql}\n)"

# 6c) Generate CREATE TABLE from CSV header + sample values
fq_tbl = f"{DB}.{SC}.supplier_case"
ddl = generate_snowflake_ddl(PG_CSV, fq_tbl, sample_rows=5000)
print("DDL:\n", ddl)
cs.execute(ddl)

# 6d) COPY into the Snowflake table
cs.execute("ALTER SESSION SET DATE_INPUT_FORMAT='AUTO', TIME_INPUT_FORMAT='AUTO', TIMESTAMP_INPUT_FORMAT='AUTO'")
cs.execute(f"""
COPY INTO {fq_tbl}
FROM @{DB}.{SC}.supplier_stage/pg_export
FILE_FORMAT=(FORMAT_NAME='{DB}.{SC}.csv_ff')
MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
PATTERN='.*supplier_case\\.csv\\.gz'
ON_ERROR='ABORT_STATEMENT'
""")

# quick check
cs.execute(f"SELECT COUNT(*) FROM {fq_tbl}")
print("supplier_case rows:", cs.fetchone()[0])
